# Commits background work

I am restarting this whole script in order to build up piece-by-piece for what I want. I am getting lost in the details of editing old code.

In [ ]:
import subprocess
from os import path
from dateutil.parser import parse as parsedate

import pandas as pd

In [ ]:
# Read the file of desired repositories.
parameters = pd.read_csv('repositories.csv',
                         index_col=False)
parameters

In [ ]:
repository_path = parameters['Repository path']
repository_path

In [ ]:
justfornow = repository_path[0]
justfornow

In [ ]:
# Create a dataframe for each repository.
history = subprocess.check_output(
    ['git', 'log', '--pretty=%aI', '--author=Gilles'], 
    cwd=path.expanduser(f'~/documents/websites/{justfornow}'),
    universal_newlines=True).splitlines()
type(history)

In [ ]:
dates = list(map(parsedate, history))
type(dates)

In [ ]:
# print(f'~/documents/websites/{repository_path}')
df = pd.DataFrame.from_dict({'Date': dates},
                            dtype='datetime64[ns]') \
                            .set_index('Date')
df.head()

In [ ]:
# df.reset_index()
df['count'] = 1
df.head()

In [ ]:
# print(df)
commits_daily = df.groupby(df.index.date).count()
# commits_daily = commits_daily[-90:].reset_index()
commits_daily = commits_daily.reset_index()\
                             .rename(columns={'index':'date'})
commits_daily.head()
# print(repository_path, commits_daily.tail())

In [ ]:
commits_daily.shape